# Data Preprocessing
In this notebook the data preprocessing form a Google Sheets file to cleaned data is shown. 

As described in the paper two of the authors went trough all 66 paper and manually labelled the reproducability variables for each paper with the help of a Google Sheets file. 




1. Read-in csv-file created in collaboration between the authors in Google Sheets
2. Clean the data and drop unnecassary columns
3. Define new index
4. add short DOI
5. reorder columns

In [1]:
import pandas as pd

In [2]:
# 1. Load Data
data = pd.read_csv(
    '../data/papers_reviewed_original.csv',
    header=1,
    usecols=lambda column: 'Unnamed' not in column,
    )

# 2. Data cleaning and column renaming
data.dropna(subset=['Paper ID old', '1a. Are all the data sources listed?'], inplace=True)
data.drop(columns=['Data Reproducability', 'Method', 'Experiment'], inplace=True)

# 3. Define new index
data['Paper ID'] = range(1, len(data) + 1)
data.set_index('Paper ID', inplace=True)

data.drop(columns=['Paper ID old'], inplace=True)

# 4. add short DOI
data['DOI_short'] = data['DOI'].str.replace('https://doi.org/', '').str.replace('https://dl.acm.org/doi/', '')


# 5. Reorder columns
first_cols = ['DOI', 'DOI_short']
columns_order = first_cols + [col for col in data.columns if col not in first_cols]
data = data[columns_order]


## Replace equal strings and sort comma-separeted values

In [3]:
# Replace equal strings
def replace_equal_strings(cell):
    if isinstance(cell, str) and '=' in cell:
        parts = cell.split('=')
        if len(parts) == 2 and parts[0].strip() == parts[1].strip():
            return parts[0].strip()
    return cell

for column in data.columns:
    if column != '4e. metrics':
        data[column] = data[column].map(replace_equal_strings)
data = data.dropna(how='all', subset=data.columns[2:])

# Sort comma-separated values
for col in ['1b. What type of data is used?', '4a. what splitting type']:
    if col in data.columns:
        data[col] = data[col].apply(lambda x: x if (not isinstance(x, str) or ',' not in str(x)) else ', '.join(sorted(str(x).split(', '))))

# Replace "not mentioned" with "No Information"
data = data.replace("not mentioned", "No Information")


In [4]:
# Special handling for '4e. metrics' column - create mapping for string replacement
metrics_mapping = {
    'Cluster related (Silhouette Index ...)': 'Cluster related', 
    'R2 Score (coeff. of determination)': 'R2 Score',
}

def clean_metrics_column(cell, mapping):
    cleaned_items = []
    if '\r\n≠\r\n' in cell:
        cell_split = cell.split('\r\n≠\r\n')
    elif '=' in cell:
        cell_split = cell.split('=')

    for element in cell_split:
        parts = element.split(', ')
        for part in parts:
                part = mapping.get(part.strip(), part.strip())
                cleaned_items.append(part.strip())

    return_str = ', '.join(sorted(set(cleaned_items)))
    
    # if "Cluster related" in cleaned_items:
    #     print(f"Original cell: {cell}")
    #     print(f"Cleaned items: {cleaned_items}")
    #     print(f"Set of cleaned items: {sorted(set(cleaned_items))}")
    #     print(f"Return string: {return_str}")
    #     print("-"* 20)
    
    return return_str

data['4e. metrics'] = data['4e. metrics'].apply(lambda x: clean_metrics_column(x, metrics_mapping))

In [5]:
data.head(5)

,DOI,DOI_short,1a. Are all the data sources listed?,1b. What type of data is used?,1c. Dataset statistics present?,1d. Data availability,1e. Benchmark Dataset,1f. Metadata/Description,2a. pre porcessing steps,2b. removed source data,...,4e. metrics,4f. stat sig tests,5a. code/workflow repo,5b. empty repo,5c. data preprocessing code,5d. feature generation code,5e. evaluation code,5f. hyperparameter tuning code,5g. supplementary info,5h. sw implementation
Paper ID,,,,,,,,,,,,,,,,,,,,,
1,https://doi.org/10.1109/CCDC.2014.6852414,10.1109/CCDC.2014.6852414,n,real-world,n,No Information,NaN,y,y,NaN,...,"Charts, Error Est.",n,n,na,na,na,na,na,na,na
2,https://doi.org/10.1109/ICTAI.2018.00136,10.1109/ICTAI.2018.00136,n,real-world,n,proprietary,NaN,y,n,NaN,...,"Precision, Recall",y,n,na,na,na,na,na,na,na
3,https://doi.org/10.1109/ICVR57957.2023.10169760,10.1109/ICVR57957.2023.10169760,n,No Information,n,No Information,NaN,n,n,NaN,...,Operational KPIs,n,n,na,na,na,na,na,na,na
4,https://doi.org/10.1109/AIKIIE60097.2023.10390401,10.1109/AIKIIE60097.2023.10390401,n,simulation,n,No Information,NaN,n,n,NaN,...,"Accuracy, F1 score, FPR, TPR",n,n,na,na,na,na,na,na,na
5,https://doi.org/10.1109/ICICT55905.2022.00043,10.1109/ICICT55905.2022.00043,n,"real-world, simulation",n,No Information,NaN,y,n,NaN,...,"Accuracy, Loss",n,n,na,na,na,na,na,na,na


## Data Mapping
Map the column name (phrases) to new column names (identifiers) according to Table 1 in the paper. 


In [6]:
# Define mappings
column_name_mapping = {
    'data_listed': ['1a. Are all the data sources listed?'],
    'data_metadata': ['1f. Metadata/Description'],
    'data_stats': ['1c. Dataset statistics present?'],
    'data_type': ['1b. What type of data is used?'],
    'data_access': ['1d. Data availability'],
    'preproc_data': ['2a. pre porcessing steps'],
    'preproc_features': ['2d. methods for features/descriptors'],
    'multiple data': ['2c. multiple sources: combination clear?'],
    'opt_mentioned': ['3a. HP opt - model(s)'],
    'opt_baseline': ['3b. HP opt - baseline(s)'],
    'opt_procedure': ['3c. HP opt procedure'],
    'params_models': ['3d. searched ranges - model'],
    'params_baselines': ['3e. searched ranges - baselines'],
    'params_best_model': ['3f. best HP - model'],
    'params_best_baseline': ['3g. best HP - baselines'],
    'eval_splitting': ['4a. what splitting type'],
    'eval_metrics': ['4e. metrics'],
    'eval_sig_test': ['4f. stat sig tests'],
    'code_link': ['5a. code/workflow repo'],
    'code_empty': ['5b.  empty repo '],
    'code_preproc': ['5c. data preprocessing code'],
    'code_features_gen': ['5d. feature generation code'],
    'code_eval': ['5e. evaluation code'],
    'code_params_opt': ['5f. hyperparameter tuning code'],
    'code_info': ['5g. supplementary info'],
    'code_runable': ['5h. sw implementation'],
}


In [7]:
# Data mapping
mapped_data = data[['DOI', 'DOI_short']].copy()

for short_name, old_name in column_name_mapping.items():
    if old_name[0] in data.columns:
        mapped_data[short_name] = data[old_name]

mapped_data.head(5)

,DOI,DOI_short,data_listed,data_metadata,data_stats,data_type,data_access,preproc_data,preproc_features,multiple data,...,eval_metrics,eval_sig_test,code_link,code_empty,code_preproc,code_features_gen,code_eval,code_params_opt,code_info,code_runable
Paper ID,,,,,,,,,,,,,,,,,,,,,
1,https://doi.org/10.1109/CCDC.2014.6852414,10.1109/CCDC.2014.6852414,n,y,n,real-world,No Information,y,y,na,...,"Charts, Error Est.",n,n,na,na,na,na,na,na,na
2,https://doi.org/10.1109/ICTAI.2018.00136,10.1109/ICTAI.2018.00136,n,y,n,real-world,proprietary,n,y,na,...,"Precision, Recall",y,n,na,na,na,na,na,na,na
3,https://doi.org/10.1109/ICVR57957.2023.10169760,10.1109/ICVR57957.2023.10169760,n,n,n,No Information,No Information,n,n,na,...,Operational KPIs,n,n,na,na,na,na,na,na,na
4,https://doi.org/10.1109/AIKIIE60097.2023.10390401,10.1109/AIKIIE60097.2023.10390401,n,n,n,simulation,No Information,n,n,na,...,"Accuracy, F1 score, FPR, TPR",n,n,na,na,na,na,na,na,na
5,https://doi.org/10.1109/ICICT55905.2022.00043,10.1109/ICICT55905.2022.00043,n,y,n,"real-world, simulation",No Information,n,y,y,...,"Accuracy, Loss",n,n,na,na,na,na,na,na,na


## Add publisher and method information

In [8]:
data_without_publisher = mapped_data.copy()
data_without_publisher['Paper ID'] = data_without_publisher.index

publisher_info = pd.read_csv('../data/unique_articles.csv')

data_with_publisher = data_without_publisher.merge(publisher_info, left_on='DOI', right_on='DOI', how='inner')

# Reorder columns starting with Paper_ID, DOI, doi, methodology, publisher
first_cols = ['Paper ID', 'DOI', 'DOI_short', 'Methodology', 'Publisher', 'Year']
column_order = first_cols + [col for col in data_with_publisher.columns if col not in first_cols]
data_with_publisher = data_with_publisher[column_order]

# Sort by 'Paper ID'
data_with_publisher.sort_values(by='Paper ID', inplace=True)
# Fill NaN values with 0 before converting to int, or handle NaN values appropriately
data_with_publisher['Paper ID'] = data_with_publisher['Paper ID'].fillna(0).astype(int)


In [9]:
# Save cleaned data to csv-file
data_with_publisher.to_csv('../data/papers_reviewed_reprod_variables.csv', index=False)